In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 19: Go With The Flow ---</h2><p>With the Elves well on their way constructing the North Pole base, you turn your attention back to understanding the inner workings of programming the device.</p>
<p>You can't help but notice that the <a href="16">device's opcodes</a> don't contain any <em>flow control</em> like jump instructions. The device's <a href="16">manual</a> goes on to explain:</p>
<p>"In programs where flow control is required, the <a href="https://en.wikipedia.org/wiki/Program_counter">instruction pointer</a> can be <em>bound to a register</em> so that it can be manipulated directly. This way, <code>setr</code>/<code>seti</code> can function as absolute jumps, <code>addr</code>/<code>addi</code> can function as relative jumps, and other opcodes can cause <span title="Good luck maintaining a program that uses a bitwise operation on its instruction pointer, though.">truly fascinating</span> effects."</p>
<p>This mechanism is achieved through a declaration like <code>#ip 1</code>, which would modify register <code>1</code> so that accesses to it let the program indirectly access the instruction pointer itself. To compensate for this kind of binding, there are now <em>six</em> registers (numbered <code>0</code> through <code>5</code>); the five not bound to the instruction pointer behave as normal. Otherwise, the same rules apply as <a href="16">the last time you worked with this device</a>.</p>
<p>When the <em>instruction pointer</em> is bound to a register, its value is written to that register just before each instruction is executed, and the value of that register is written back to the instruction pointer immediately after each instruction finishes execution. Afterward, move to the next instruction by adding one to the instruction pointer, even if the value in the instruction pointer was just updated by an instruction. (Because of this, instructions must effectively set the instruction pointer to the instruction <em>before</em> the one they want executed next.)</p>
<p>The instruction pointer is <code>0</code> during the first instruction, <code>1</code> during the second, and so on. If the instruction pointer ever causes the device to attempt to load an instruction outside the instructions defined in the program, the program instead immediately halts. The instruction pointer starts at <code>0</code>.</p>
<p>It turns out that this new information is already proving useful: the CPU in the device is not very powerful, and a background process is occupying most of its time.  You dump the background process' declarations and instructions to a file (your puzzle input), making sure to use the names of the opcodes rather than the numbers.</p>
<p>For example, suppose you have the following program:</p>
<pre><code>#ip 0
seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5
</code></pre>
<p>When executed, the following instructions are executed. Each line contains the value of the instruction pointer at the time the instruction started, the values of the six registers before executing the instructions (in square brackets), the instruction itself, and the values of the six registers after executing the instruction (also in square brackets).</p>
<pre><code>ip=0 [0, 0, 0, 0, 0, 0] seti 5 0 1 [0, 5, 0, 0, 0, 0]
ip=1 [1, 5, 0, 0, 0, 0] seti 6 0 2 [1, 5, 6, 0, 0, 0]
ip=2 [2, 5, 6, 0, 0, 0] addi 0 1 0 [3, 5, 6, 0, 0, 0]
ip=4 [4, 5, 6, 0, 0, 0] setr 1 0 0 [5, 5, 6, 0, 0, 0]
ip=6 [6, 5, 6, 0, 0, 0] seti 9 0 5 [6, 5, 6, 0, 0, 9]
</code></pre>
<p>In detail, when running this program, the following events occur:</p>
<ul>
<li>The first line (<code>#ip 0</code>) indicates that the instruction pointer should be bound to register <code>0</code> in this program. This is not an instruction, and so the value of the instruction pointer does not change during the processing of this line.</li>
<li>The instruction pointer contains <code>0</code>, and so the first instruction is executed (<code>seti 5 0 1</code>).  It updates register <code>0</code> to the current instruction pointer value (<code>0</code>), sets register <code>1</code> to <code>5</code>, sets the instruction pointer to the value of register <code>0</code> (which has no effect, as the instruction did not modify register <code>0</code>), and then adds one to the instruction pointer.</li>
<li>The instruction pointer contains <code>1</code>, and so the second instruction, <code>seti 6 0 2</code>, is executed. This is very similar to the instruction before it: <code>6</code> is stored in register <code>2</code>, and the instruction pointer is left with the value <code>2</code>.</li>
<li>The instruction pointer is <code>2</code>, which points at the instruction <code>addi 0 1 0</code>.  This is like a <em>relative jump</em>: the value of the instruction pointer, <code>2</code>, is loaded into register <code>0</code>. Then, <code>addi</code> finds the result of adding the value in register <code>0</code> and the value <code>1</code>, storing the result, <code>3</code>, back in register <code>0</code>. Register <code>0</code> is then copied back to the instruction pointer, which will cause it to end up <code>1</code> larger than it would have otherwise and skip the next instruction (<code>addr 1 2 3</code>) entirely. Finally, <code>1</code> is added to the instruction pointer.</li>
<li>The instruction pointer is <code>4</code>, so the instruction <code>setr 1 0 0</code> is run. This is like an <em>absolute jump</em>: it copies the value contained in register <code>1</code>, <code>5</code>, into register <code>0</code>, which causes it to end up in the instruction pointer. The instruction pointer is then incremented, leaving it at <code>6</code>.</li>
<li>The instruction pointer is <code>6</code>, so the instruction <code>seti 9 0 5</code> stores <code>9</code> into register <code>5</code>. The instruction pointer is incremented, causing it to point outside the program, and so the program ends.</li>
</ul>
<p><em>What value is left in register <code>0</code></em> when the background process halts?</p>
</article>


In [2]:
from re import match

from more_itertools import last, tail, take


example = """
#ip 0
seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5
"""


class Device:
    def __init__(self, number_of_registers: int = 6) -> None:
        self.ip = 0
        self.registers = [0] * number_of_registers
        self.instructions = []

    def load(self, program: str) -> Device:
        self.instructions = [
            [int(d) if match(r"\d+", d) else d for d in line.split()]
            for line in program.strip().splitlines()
        ]
        self.ip = last(self.instructions.pop(0))

        return self

    def process(self, do_print: bool = False) -> Device:
        while 0 <= self.registers[self.ip] < len(self.instructions):
            self.process_line(self.registers[self.ip], do_print)
            self.registers[self.ip] += 1
        self.registers[self.ip] -= 1
        return self

    def process_II(self, do_print: bool = False) -> Device:
        while 0 <= self.registers[self.ip] < len(self.instructions):
            self.process_line(self.registers[self.ip], do_print)

            if self.registers[self.ip] == 32:
                n = sum(tail(2, sorted(self.registers)))
                self.registers[0] = sum(i for i in range(1, n + 1) if n % i == 0)
                return self

            self.registers[self.ip] += 1
        self.registers[self.ip] -= 1
        return self

    def process_line(self, ip: int, do_print: bool) -> None:
        inst, A, B, C = self.instructions[ip]
        # Addition:
        # addr (add register) stores into register C the result of
        # adding register A and register B.
        if do_print:
            s = f"{self} {inst} {A} {B} {C}"

        if inst == "addr":
            self.registers[C] = self.registers[A] + self.registers[B]
        # addi (add immediate) stores into register C the result of
        # adding register A and value B.
        elif inst == "addi":
            self.registers[C] = self.registers[A] + B

        # Multiplication:
        # mulr (multiply register) stores into register C the result
        # of multiplying register A and register B.
        elif inst == "mulr":
            self.registers[C] = self.registers[A] * self.registers[B]
        # muli (multiply immediate) stores into register C the result
        # of multiplying register A and value B.
        elif inst == "muli":
            self.registers[C] = self.registers[A] * B

        # Bitwise AND:
        # banr (bitwise AND register) stores into register C the result
        # of the bitwise AND of register A and register B.
        elif inst == "banr":
            self.registers[C] = self.registers[A] & self.registers[B]
        # bani (bitwise AND immediate) stores into register C the result
        # of the bitwise AND of register A and value B.
        elif inst == "bani":
            self.registers[C] = self.registers[A] & B

        # Bitwise OR:
        # borr (bitwise OR register) stores into register C the result
        # of the bitwise OR of register A and register B.
        elif inst == "borr":
            self.registers[C] = self.registers[A] | self.registers[B]
        # bori (bitwise OR immediate) stores into register C the result
        # of the bitwise OR of register A and value B.
        elif inst == "bori":
            self.registers[C] = self.registers[A] | B

        # Assignment:
        # setr (set register) copies the contents of register A into register C.
        # (Input B is ignored.)
        elif inst == "setr":
            self.registers[C] = self.registers[A]
        # seti (set immediate) stores value A into register C. (Input B is ignored.)
        elif inst == "seti":
            self.registers[C] = A

        # Greater-than testing:
        # gtir (greater-than immediate/register)
        # sets register C to 1 if value A is greater than register B.
        # Otherwise, register C is set to 0.
        elif inst == "gtir":
            self.registers[C] = 1 if A > self.registers[B] else 0

        # gtri (greater-than register/immediate) sets register C to 1
        # if register A is greater than value B. Otherwise, register C is set to 0.
        elif inst == "gtri":
            self.registers[C] = 1 if self.registers[A] > B else 0
        # gtrr (greater-than register/register) sets register C to 1
        # if register A is greater than register B. Otherwise, register C
        # is set to 0.
        elif inst == "gtrr":
            self.registers[C] = 1 if self.registers[A] > self.registers[B] else 0

        # Equality testing:
        # eqir (equal immediate/register) sets register C to 1
        # if value A is equal to register B. Otherwise, register C is set to 0.
        elif inst == "eqir":
            self.registers[C] = 1 if A == self.registers[B] else 0
        # eqri (equal register/immediate) sets register C to 1
        # if register A is equal to value B. Otherwise, register C is set to 0.
        elif inst == "eqri":
            self.registers[C] = 1 if self.registers[A] == B else 0
        # eqrr (equal register/register) sets register C to 1
        # if register A is equal to register B. Otherwise, register C is set to 0.
        elif inst == "eqrr":
            self.registers[C] = 1 if self.registers[A] == self.registers[B] else 0

        if do_print:
            print(f"{s} {self.registers}")

    def get_ip_register(self) -> int:
        return self.registers[self.ip]

    def get_register(self, nr: int) -> int:
        return self.registers[nr]

    def set_register(self, nr: int, value: int) -> Device:
        self.registers[nr] = value
        return self

    def stringify_program(self) -> str:
        return "\n".join(
            f"{i:03d} {self.stringify_program_line(i)}"
            for i in range(len(self.instructions))
        )

    def stringify_program_line(self, ip: int) -> str:
        inst, A, B, C = self.instructions[ip]
        # Addition:
        # addr (add register) stores into register C the result of
        # adding register A and register B.

        if inst == "addr":
            return f"register[{C}] = register[{A}] + register[{B}]"
        # addi (add immediate) stores into register C the result of
        # adding register A and value B.
        if inst == "addi":
            return f"register[{C}] = register[{A}] + {B}"

        # Multiplication:
        # mulr (multiply register) stores into register C the result
        # of multiplying register A and register B.
        if inst == "mulr":
            return f"register[{C}] = register[{A}] * register[{B}]"
        # muli (multiply immediate) stores into register C the result
        # of multiplying register A and value B.
        if inst == "muli":
            return f"register[{C}] = register[{A}] * {B}"

        # Bitwise AND:
        # banr (bitwise AND register) stores into register C the result
        # of the bitwise AND of register A and register B.
        if inst == "banr":
            return f"register[{C}] = register[{A}] & register[{B}]"
        # bani (bitwise AND immediate) stores into register C the result
        # of the bitwise AND of register A and value B.
        if inst == "bani":
            return f"register[{C}] = register[{A}] & {B}"

        # Bitwise OR:
        # borr (bitwise OR register) stores into register C the result
        # of the bitwise OR of register A and register B.
        if inst == "borr":
            return f"register[{C}] = register[{A}] | register[{B}]"
        # bori (bitwise OR immediate) stores into register C the result
        # of the bitwise OR of register A and value B.
        if inst == "bori":
            return f"register[{C}] = register[{A}] | {B}"

        # Assignment:
        # setr (set register) copies the contents of register A into register C.
        # (Input B is ignored.)
        if inst == "setr":
            return f"register[{C}] = register[{A}]"
        # seti (set immediate) stores value A into register C. (Input B is ignored.)
        if inst == "seti":
            return f"register[{C}] = {A}"

        # Greater-than testing:
        # gtir (greater-than immediate/register)
        # sets register C to 1 if value A is greater than register B.
        # Otherwise, register C is set to 0.
        if inst == "gtir":
            return f"register[{C}] = 1 if {A} > register[{B}] else 0"

        # gtri (greater-than register/immediate) sets register C to 1
        # if register A is greater than value B. Otherwise, register C is set to 0.
        if inst == "gtri":
            return f"register[{C}] = 1 if register[{A}] > {B} else 0"
        # gtrr (greater-than register/register) sets register C to 1
        # if register A is greater than register B. Otherwise, register C
        # is set to 0.
        if inst == "gtrr":
            return f"register[{C}] = 1 if register[{A}] > register[{B}] else 0"

        # Equality testing:
        # eqir (equal immediate/register) sets register C to 1
        # if value A is equal to register B. Otherwise, register C is set to 0.
        if inst == "eqir":
            return f"register[{C}] = 1 if A == register[{B}] else 0"
        # eqri (equal register/immediate) sets register C to 1
        # if register A is equal to value B. Otherwise, register C is set to 0.
        if inst == "eqri":
            return f"register[{C}] = 1 if register[{A}] == {B} else 0"
        # eqrr (equal register/register) sets register C to 1
        # if register A is equal to register B. Otherwise, register C is set to 0.
        if inst == "eqrr":
            return f"register[{C}] = 1 if register[{A}] == register[{B}] else 0"

        raise ValueError(f"Instruction line {ip}={self.instructions[ip]} unknown")

    def __repr__(self) -> str:
        return f"{self.ip=} {self.registers}"


print(f"Example: {Device().load(example).process().get_register(0)}")

Example: 6


In [3]:
with open("../input/day19.txt") as f:
    puzzle = f.read()

print(f"Part I: {Device().load(puzzle).process_II().get_register(0)}")

Part I: 1488


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1488</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>A new background process immediately spins up in its place. It appears identical, but on closer inspection, you notice that <em>this time, register <code>0</code> started with the value <code>1</code></em>.</p>
<p><em>What value is left in register <code>0</code></em> when this new background process halts?</p>
</article>

</main>


In [4]:
# #ip 2
# 00  addi 2 16 2
# 01  seti 1 1 3
# 02  seti 1 7 5
# 03  mulr 3 5 4
# 04  eqrr 4 1 4
# 05  addr 4 2 2
# 06  addi 2 1 2
# 07  addr 3 0 0
# 08  addi 5 1 5
# 09  gtrr 5 1 4
# 10  addr 2 4 2
# 11  seti 2 3 2
# 12  addi 3 1 3
# 13  gtrr 3 1 4
# 14  addr 4 2 2
# 15  seti 1 9 2
# 16  mulr 2 2 2
# 17  addi 1 2 1
# 18  mulr 1 1 1
# 19  mulr 2 1 1
# 20  muli 1 11 1
# 21  addi 4 3 4
# 22  mulr 4 2 4
# 23  addi 4 13 4
# 24  addr 1 4 1
# 25  addr 2 0 2
# 26  seti 0 1 2
# 27  setr 2 0 4
# 28  mulr 4 2 4
# 29  addr 2 4 4
# 30  mulr 2 4 4
# 31  muli 4 14 4
# 32  mulr 4 2 4 <- set value for factors
# 33  addr 1 4 1 <- jumps back to 4
# 34  seti 0 4 0
#   seti 0 5 2
print(Device().load(puzzle).stringify_program())
print()
print(
    f"Part II: {Device().load(puzzle).set_register(0, 1).process_II().get_register(0)}"
)

000 register[2] = register[2] + 16
001 register[3] = 1
002 register[5] = 1
003 register[4] = register[3] * register[5]
004 register[4] = 1 if register[4] == register[1] else 0
005 register[2] = register[4] + register[2]
006 register[2] = register[2] + 1
007 register[0] = register[3] + register[0]
008 register[5] = register[5] + 1
009 register[4] = 1 if register[5] > register[1] else 0
010 register[2] = register[2] + register[4]
011 register[2] = 2
012 register[3] = register[3] + 1
013 register[4] = 1 if register[3] > register[1] else 0
014 register[2] = register[4] + register[2]
015 register[2] = 1
016 register[2] = register[2] * register[2]
017 register[1] = register[1] + 2
018 register[1] = register[1] * register[1]
019 register[1] = register[2] * register[1]
020 register[1] = register[1] * 11
021 register[4] = register[4] + 3
022 register[4] = register[4] * register[2]
023 register[4] = register[4] + 13
024 register[1] = register[1] + register[4]
025 register[2] = register[2] + regi

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>17427456</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
